# Multi-Feature RSP Visualization

This script creates visualizations of RSP (Radar Scanning Plot) using KPMP data for multiple genes simultaneously. You can use this to check if the package is working correctly and to compare spatial expression patterns between different genes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np

In [ ]:
import spatialrsp as rsp

In [ ]:
import logging

logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s %(name)s %(levelname)-8s %(message)s"
)

In [ ]:
plt.rcParams["figure.dpi"] = 400
plt.rcParams["font.size"] = 14
plt.rcParams["axes.grid"] = True

distinct_colors = [
    "#d62728",
    "#2ca02c",
    "#ff7f0e",
    "#1f77b4",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#bcbd22",
    "#17becf",
]

In [ ]:
path = rsp.utils.download_kpmp(
    variant="sn",
    # force=True, # Uncomment to force re-download; useful if you think the data was contaminated
)
print("Downloaded KPMP data to", path)

In [ ]:
adata = rsp.io.load_data(path)

In [ ]:
preprocessor = rsp.Preprocessor()

In [ ]:
tal_cells = adata[adata.obs["subclass.l1"] == "TAL"].copy()
logging.info(f"TAL cells shape: {tal_cells.shape}")

In [ ]:
ctal_cells = tal_cells[tal_cells.obs["subclass.l2"] == "C-TAL"].copy()
logging.info(f"C-TAL cells shape: {ctal_cells.shape}")

In [ ]:
genes = ["UMOD", "PROM1"]  # Select genes of interest
threshold_percentile = 0.5  # Select top x% of expressing cells
filtered_cells = tal_cells
scanning_range = np.linspace(0, 2 * np.pi, 360)
resolution = 100

In [ ]:
preprocessor.run(
    filtered_cells,
    qc=False,
    normalize=False,
    reduction=None,
    polar=True,
)

In [ ]:
def match_gene_name(adata, gene):
    if "feature_name" in adata.var.columns:
        matched_genes = adata.var[
            adata.var["feature_name"].str.contains(gene, case=False)
        ]
        if not matched_genes.empty:
            return matched_genes.index[0]
    raise ValueError(f"Gene '{gene}' not found in the dataset.")

In [ ]:
ensgenes = {}
thresholds = {}
expressions = {}

for gene in genes:
    try:
        ensgenes[gene] = match_gene_name(filtered_cells, gene)
        logging.info(f"Matched {gene} to {ensgenes[gene]}")
    except ValueError as e:
        logging.error(e)
        continue

    expression = filtered_cells[:, ensgenes[gene]].X.toarray().flatten()
    expressions[gene] = expression

    thresholds[gene] = rsp.utils.percentile_to_threshold(
        expressions[gene], threshold_percentile
    )
    logging.info(
        f"Threshold for {gene} at {threshold_percentile * 100}%: {thresholds[gene]}"
    )

In [ ]:
print("=== Expression Data Storage Summary ===")
print(f"Stored expression data for {len(expressions)} genes:")
for gene in expressions:
    expr_array = expressions[gene]
    non_zero_cells = np.sum(expr_array > 0)
    mean_expr = np.mean(expr_array[expr_array > 0]) if non_zero_cells > 0 else 0
    max_expr = np.max(expr_array)

    print(f"{gene}:")
    print(f"  - Array shape: {expr_array.shape}")
    print(
        f"  - Non-zero cells: {non_zero_cells}/{len(expr_array)} ({non_zero_cells/len(expr_array)*100:.1f}%)"
    )
    print(f"  - Mean expression (non-zero): {mean_expr:.3f}")
    print(f"  - Max expression: {max_expr:.3f}")
    print()

In [ ]:
fg_masks = {}
fg_angles_list = []

In [ ]:
bg_angles = rsp.utils.get_polar_angles(
    adata=filtered_cells,
    mask=None,  # Background = all cells
    polar_coord="X_polar",
)

In [ ]:
for gene in genes:
    ensgene = ensgenes[gene]
    gene_threshold = thresholds[gene]

    fg_mask = rsp.utils.extract_foreground_mask(
        adata=filtered_cells,
        feature=ensgene,
        threshold=gene_threshold,
    )
    fg_mask = fg_mask.astype(bool)

    # check if mask is a boolean array
    if not np.issubdtype(fg_mask.dtype, np.bool_):
        raise ValueError(f"Foreground mask for {gene} is not a boolean array.")
    else:
        logging.info(f"Foreground mask for {gene} is a boolean array.")
        logging.info(
            f"Selected {np.sum(fg_mask)} out of {len(fg_mask)} cells for {gene} "
            f"({np.sum(fg_mask)/len(fg_mask)*100:.1f}%)"
        )

    fg_masks[gene] = fg_mask

    fg_angles = rsp.utils.get_polar_angles(
        adata=filtered_cells,
        mask=fg_mask,
        polar_coord="X_polar",
    )
    fg_angles_list.append(fg_angles)

In [ ]:
fg_curves, exp_curves, bg_curve = rsp.compute_rsp(
    theta_fgs=fg_angles_list,
    theta_bg=bg_angles,
    scanning_window=np.pi,
    scanning_range=scanning_range,
    resolution=resolution,
    mode="absolute",
)

In [ ]:
# Display threshold summary
print("=== Threshold Summary ===")
print(
    f"Threshold percentile: {threshold_percentile} (selecting top {(1-threshold_percentile)*100:.0f}% of cells)"
)
print()
for gene in genes:
    threshold_val = thresholds[gene]
    fg_mask = fg_masks[gene]
    selected_cells = np.sum(fg_mask)
    
    # Use stored expression data instead of re-extracting
    is_expressed = expressions[gene] > 0
    total_cells = np.sum(is_expressed)
    selected_pct = selected_cells / total_cells * 100

    print(f"{gene}:")
    print(f"  - Actual threshold: {threshold_val:.3f}")
    print(f"  - Selected cells: {selected_cells}/{total_cells} ({selected_pct:.1f}%)")
    print()

In [ ]:
fig = plt.figure(figsize=(12, 6))
gs = gridspec.GridSpec(1, 2, figure=fig, width_ratios=[10, 7])
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], projection="polar")

colors = distinct_colors[: len(genes)]

umap_coords = filtered_cells.obsm["X_umap"]
ax1.scatter(
    umap_coords[:, 0], umap_coords[:, 1], c="gray", s=1, alpha=0.3, label="Background"
)

for i, gene in enumerate(genes):
    fg_mask = fg_masks[gene]
    threshold_val = thresholds[gene]
    selected_pct = np.sum(fg_mask) / len(fg_mask) * 100

    ax1.scatter(
        umap_coords[fg_mask, 0],
        umap_coords[fg_mask, 1],
        c=colors[i],
        s=2,
        alpha=0.8,
        label=f"{gene} (thr: {threshold_val:.2f})",
    )

ax1.legend(loc="upper right", fontsize=10)
title_text = f"UMAP (Top {(1-threshold_percentile)*100:.0f}% cells selected)"
ax1.set_title(title_text, fontsize=14, pad=15)
ax1.set_xlabel("UMAP1", fontsize=14)
ax1.set_ylabel("UMAP2", fontsize=14)
ax1.tick_params(labelsize=12)
ax1.set_aspect("equal")

theta = np.asarray(scanning_range)
n = len(fg_curves[0])

if theta.size == n + 1 and np.isclose((theta[-1] - theta[0]) % (2 * np.pi), 0.0):
    theta = theta[:-1]
elif theta.size == 2:
    start, end = theta
    theta = np.linspace(start, end, n, endpoint=False)
elif theta.size != n:
    raise ValueError(f"scanning_range length {theta.size} but fg_curve length {n}")

theta_closed = np.concatenate([theta, [theta[0]]])
bg_closed = np.concatenate([bg_curve, [bg_curve[0]]])

ax2.plot(theta_closed, bg_closed, ":", c="darkgray", label="Background", linewidth=1)

for i, gene in enumerate(genes):
    fg_curve = fg_curves[i]
    exp_curve = exp_curves[i]

    fg_closed = np.concatenate([fg_curve, [fg_curve[0]]])
    exp_closed = np.concatenate([exp_curve, [exp_curve[0]]])

    ax2.plot(
        theta_closed,
        fg_closed,
        c=colors[i],
        alpha=0.8,
        label=f"{gene} (observed)",
        linewidth=2,
        linestyle="-",
    )

    ax2.plot(
        theta_closed,
        exp_closed,
        c=colors[i],
        alpha=0.6,
        label=f"{gene} (expected)",
        linewidth=1.5,
        linestyle="--",
    )

ax2.set_title(f"RSP (Absolute Mode)", fontsize=14, pad=20)
ax2.legend(loc="lower right", bbox_to_anchor=(1.1, -0.22), fontsize=10)
ax2.tick_params(labelsize=12)

plt.tight_layout()

## Interpretation

The visualization above shows:

1. **Left panel (UMAP)**: The spatial distribution of cells expressing each gene above the percentile threshold
   - Gray dots: Background cells (low expression)
   - Colored dots: Cells with high expression for each gene
   - Legend shows the actual threshold value for each gene
2. **Right panel (RSP - Absolute Mode)**: The radar scanning plot showing angular enrichment patterns
   - **Solid lines**: Observed spatial enrichment for each gene
   - **Dashed lines**: Expected enrichment under the local background model
   - **Dotted gray line**: Background reference curve
   - Values represent absolute spatial clustering strength

### RSP Curve Interpretation

- **Observed > Expected**: Gene shows stronger spatial clustering than expected
- **Observed < Expected**: Gene shows weaker spatial clustering than expected
- **Observed ≈ Expected**: Gene follows expected spatial distribution patterns
- **Peak directions**: Indicate preferred spatial orientations for gene expression
